# **Stock Price Prediction using LSTM in Python**

**This notebook demonstrates how to predict stock prices using Long Short-Term Memory (LSTM) neural networks. We will use historical stock prices of Apple Inc. (AAPL) over a 10-year period.**

**Importing Required Libraries :**

First, we import all the necessary libraries.

In [ ]:
# Disable GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

!pip install yfinance


# Importing Libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')


**Function to Fetch Stock Data :**

We create a function to fetch the historical stock data from Yahoo Finance using the yfinance library.

In [ ]:
def stock_data(stock, start_date, end_date):
    tickerSymbol = stock
    tickerData = yf.Ticker(tickerSymbol)
    tickerDf = tickerData.history(period='1d', start=start_date, end=end_date)
    return tickerDf

df = stock_data("AAPL", "2014-06-08", "2027-06-08")
df.head()

**Plotting Fetched Data :**

We plot the closing price history of the stock.

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.xlabel("Year")
plt.ylabel("Closing Price in USD")
plt.plot(df['Close'], label='Close Price history')
plt.legend()
plt.show()


**Checking Data Types and Descriptive Statistics :**

We check the data types of each column and descriptive statistics to ensure data consistency.

In [ ]:
df.dtypes
df.describe()


**Resetting Index and Dropping Unnecessary Columns :**

We reset the index to make the date a column and drop columns that are not required for our analysis.

In [ ]:
df.reset_index(level="Date", inplace=True)
df.drop(columns=['Dividends', 'Stock Splits'], inplace=True)


**Creating Dataset with Closing Price :**

We create a dataset with only the closing price column and set the date as the index.

In [ ]:
data = df.sort_index(ascending=True, axis=0)
df1 = pd.DataFrame(index=range(0, len(df)), columns=['Date', 'Close'])
for i in range(0, len(data)):
    df1["Date"][i] = data['Date'][i]
    df1["Close"][i] = data["Close"][i]

df1.index = df1.Date
df1.drop("Date", axis=1, inplace=True)
df1.head()


**Splitting Data into Train and Test Sets :**

We split the data into training and testing sets.

In [ ]:
data = df1.values
train = data[0:1750, :]
test = data[1750:, :]

print(train.shape)
print(test.shape)


**Normalizing the Data :**

We normalize the data to the range of 0-1 using MinMaxScaler.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


**Preparing Training Data :**

We prepare the training data by creating sequences of 60 days to predict the next closing price.

In [ ]:
x_train, y_train = [], []
window = 60

for i in range(window, len(train)):
    x_train.append(scaled_data[i-window:i, 0])
    y_train.append(scaled_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


**Preparing Test Data :**

We prepare the test data in a similar manner to the training data.

In [ ]:
inputs = df1[len(df1) - len(test) - window:].values
inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)

x_test = []
for i in range(window, inputs.shape[0]):
    x_test.append(inputs[i-window:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


**Building the LSTM Model :**

We build and compile the LSTM model.

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()


**Training the Model :**

We train the model on the training data.

In [ ]:
model.fit(x_train, y_train, epochs=2, batch_size=64, verbose=2)


**Making Predictions :**

We make predictions on the test data and transform the results back to the original form.

In [ ]:
closing_price = model.predict(x_test)
train_predict = model.predict(x_train)

closing_price = scaler.inverse_transform(closing_price)
train_predict = scaler.inverse_transform(train_predict)


**Calculating Performance Metric :**

We calculate the Mean Squared Error (MSE) on the test data.

In [ ]:
mse = math.sqrt(mean_squared_error(test, closing_price))
print(f"Root Mean Squared Error: {mse}")


**Plotting the Predictions :**

We plot the actual and predicted closing prices.

In [ ]:
train = df1[:1750]
test = df1[1750:]
test['Predictions'] = closing_price

plt.figure(figsize=(17,8))
plt.xlabel("Year")
plt.ylabel("Closing Price in USD")
plt.title("CLOSING PRICE PREDICTION")
plt.plot(train['Close'])
plt.plot(test[['Close', 'Predictions']])
plt.legend(["Train Data", "Actual Price", "Predicted Price"], loc="lower right")
plt.show()


**The code provides a solid foundation for stock price prediction using LSTM. 
While it has potential applications, achieving robust and reliable predictions for trading purposes would require more advanced techniques, extensive data, and thorough validation.**